<center><img src="https://i.imgur.com/zRrFdsf.png" width="700"></center> 

_____

<a id='home'></a>

# Appending

<a target="_blank" href="https://colab.research.google.com/github/CienciaDeDatosEspacial/code_and_data/blob/main/Appending.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

<a id='appending'></a>

As the name implies, this process binds DFs into one, that is, one or more DFs will be put below or on top of another DF. Appending can be done when you fulfill these requisites:
1. All the DFs  share the same column names.
2. All the DFs  columns are in the same location.

Note that it is better if the columns share the same data types. But you can solve it during the formatting process.


Let's visit this website: https://fundforpeace.org/what-we-do/country-risk-and-fragility-data/

There, you will find several excel files with the _Fragile States Index_ per year. I have the files from 2013 to 2021 in a GitHub repo:

![](fragilityGit.png)

Let's read every file. For that, we will use a link to each. Let's do it step by step:

In [1]:
# link to repo - common to all files
dataRepo='https://github.com/enrique1157/datasemana7/raw/main/excels2013to2023'

In [2]:
# creating file names into a list:
years=range(2013,2022)
fileNames=['fsi-'+str(year)+'.xlsx' for year in years]
# list of file names
fileNames

['fsi-2013.xlsx',
 'fsi-2014.xlsx',
 'fsi-2015.xlsx',
 'fsi-2016.xlsx',
 'fsi-2017.xlsx',
 'fsi-2018.xlsx',
 'fsi-2019.xlsx',
 'fsi-2020.xlsx',
 'fsi-2021.xlsx']

In [3]:
# creating the url to each file:
alltheLinks=[dataRepo+fn for fn in fileNames]
alltheLinks

['https://github.com/enrique1157/datasemana7/raw/main/excels2013to2023fsi-2013.xlsx',
 'https://github.com/enrique1157/datasemana7/raw/main/excels2013to2023fsi-2014.xlsx',
 'https://github.com/enrique1157/datasemana7/raw/main/excels2013to2023fsi-2015.xlsx',
 'https://github.com/enrique1157/datasemana7/raw/main/excels2013to2023fsi-2016.xlsx',
 'https://github.com/enrique1157/datasemana7/raw/main/excels2013to2023fsi-2017.xlsx',
 'https://github.com/enrique1157/datasemana7/raw/main/excels2013to2023fsi-2018.xlsx',
 'https://github.com/enrique1157/datasemana7/raw/main/excels2013to2023fsi-2019.xlsx',
 'https://github.com/enrique1157/datasemana7/raw/main/excels2013to2023fsi-2020.xlsx',
 'https://github.com/enrique1157/datasemana7/raw/main/excels2013to2023fsi-2021.xlsx']

In [4]:
#una lista de dataframes es el allDFs

We will save each data frame in a list **allDFs**. We will use pandas, but we need **openpyxl** and **xlrd** (for Excel) before doing this operation:

In [5]:
# creating list of DFs
import pandas as pd

allDFs=[pd.read_excel(link) for link in alltheLinks] 


HTTPError: HTTP Error 404: Not Found

In [ ]:
# saving column names
allColumnNames=[]
for df in allDFs:
    allColumnNames.append(set(df.columns))# list of sets!

In [ ]:
# checking how many columns per df

[len(cols) for cols in allColumnNames]

We have an extra column in a couple of years. 
Let's find the common columns:

In [ ]:
# details of common columns
commonColumns=set.intersection(*allColumnNames) # expanding list of sets (*)
len(commonColumns),commonColumns

These are the columns not in the common names:

In [ ]:
# all minus the common
set.union(*allColumnNames)-commonColumns

We could make a list of data frames with only the common columns:

In [ ]:
# DFs with the common columns
allDFs_sameNames=[df.loc[:,list(commonColumns)] for df in allDFs]

Appending in pandas requires a list of data frames, in these case that is **allDFs_sameNames**. Then we proceed:

In [6]:
# appending
allDFsConcat=pd.concat(allDFs_sameNames)
allDFsConcat.head()

NameError: name 'allDFs_sameNames' is not defined

We could pay attention to the current data types:

In [15]:
allDFsConcat.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1603 entries, 0 to 178
Data columns (total 16 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Year                              1603 non-null   object 
 1   P3: Human Rights                  1603 non-null   float64
 2   Rank                              1603 non-null   object 
 3   Country                           1603 non-null   object 
 4   C1: Security Apparatus            1603 non-null   float64
 5   C2: Factionalized Elites          1603 non-null   float64
 6   E1: Economy                       1603 non-null   float64
 7   Total                             1603 non-null   float64
 8   P2: Public Services               1603 non-null   float64
 9   S1: Demographic Pressures         1603 non-null   float64
 10  X1: External Intervention         1603 non-null   float64
 11  E3: Human Flight and Brain Drain  1603 non-null   float64
 12  C3: Group Gr

The columns] **Year** was expected to be a numeric type, but we got an _object_ instead. Let's explore that column:

In [16]:
# exploring year column as frequency table
allDFsConcat.Year.value_counts()

Year
2021                   179
2013-01-01 00:00:00    178
2014-01-01 00:00:00    178
2015-01-01 00:00:00    178
2016-01-01 00:00:00    178
2017-01-01 00:00:00    178
2018-01-01 00:00:00    178
2019-01-01 00:00:00    178
2020-01-01 00:00:00    178
Name: count, dtype: int64

Except for the year 2021, the other values are in date-time format. We just need an integer number, then:

In [ ]:
# keeping just the year value
yearAsNumber=[]
for y in allDFsConcat.Year:
    try:
        yearAsNumber.append(y.year)# the value from a date-time format
    except:
        yearAsNumber.append(y) # if not a datetime

#verifying
pd.Series(yearAsNumber).value_counts()

In [ ]:
# overwriting the year column
allDFsConcat.Year=yearAsNumber

You may have notice that the column ordering does not look appropriate. In general you expect that the columns to the left start with identification of the rows rather than some measurements. Let's move 'Country','Year','Total' to the left:

In [ ]:
# this is a trick: setting columns as index
allDFsConcat.set_index(['Country','Year','Total'],inplace=True)
allDFsConcat.head()

Since I will not use _Rank_, I will get rid of it:

In [ ]:
# dropping unneeded column
allDFsConcat.drop(columns='Rank',inplace=True)

Let's order the current column names:

In [ ]:
# ordering column names alphabetically
allDFsConcat.sort_index(axis=1,inplace=True) # by row index will be axis=0

Now put the row indexes back:

In [ ]:
# indexes will be columns
allDFsConcat.reset_index(inplace=True)

Let's do some cleaning on the column names:

In [ ]:
# see column names
allDFsConcat.columns.to_list()

In [ ]:
# clean column names
allDFsConcat.columns=allDFsConcat.columns.str.replace(':\s',"_",regex=True)
allDFsConcat.columns=allDFsConcat.columns.str.replace('\s',"",regex=True)
#see
allDFsConcat.columns.to_list()

Let's set the country names into upper case:

In [ ]:
# overwriting country
allDFsConcat.Country=allDFsConcat.Country.str.upper()

Finally, let's check the format:

In [ ]:
allDFsConcat.info()

We should save this result:

In [ ]:
import os

allDFsConcat.to_csv(os.path.join("data","Fragility.csv"),index=False)